# -1)  imports

In [106]:
import sklearn
import pandas           as     pd
from   sklearn.datasets import load_boston


# 0) récupération de data

In [107]:
boston = load_boston()
boston.DESCR # description
X = pd.DataFrame(boston.data)
Y = pd.DataFrame(boston.target)
X.columns = boston.feature_names
X.shape, Y.shape

((506, 13), (506, 1))

# 1) division en jeu de train / test / validation

In [108]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
print("80% = ", X_train.shape,  Y_train.shape)
print("20% = ", X_test.shape,Y_test.shape)

80% =  (404, 13) (404, 1)
20% =  (102, 13) (102, 1)


## 1.2) création d'un jeu de validation

In [109]:
X_test, X_validation, Y_test, Y_validation = train_test_split(X_test, Y_test, test_size=0.2)
print("X train  / Y train =" , X_train.shape, Y_train.shape)
print("X test  / Y test   =" , X_test.shape, Y_test.shape)
print("X valid / Y valid  =", X_validation.shape, Y_validation.shape)

X train  / Y train = (404, 13) (404, 1)
X test  / Y test   = (81, 13) (81, 1)
X valid / Y valid  = (21, 13) (21, 1)


# 2) modélisation de Y par X

In [138]:
import numpy as np

array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])

In [143]:
from sklearn import linear_model
algos = { "lasso"      : linear_model.Lasso()         ,
          "elasticnet" : linear_model.ElasticNet()    , 
          "ridge_alpha"      : linear_model.Ridge(),
        }
for alpha in np.linspace(0,1,10):
    algos["ridge_alpha_{alpha}".format(alpha=alpha)] = linear_model.Ridge(alpha=alpha)

results = {}
for nom_algo in algos.keys():    
    algorithme        = algos[nom_algo]
    
    modele            = algorithme.fit(X_train, Y_train)
    score             = modele.score(X_test, Y_test)
    results[nom_algo] = score
    
pd.DataFrame([results]).T

,0
elasticnet,0.556643
lasso,0.549983
ridge_alpha,0.610058
ridge_alpha_0.0,0.615142
ridge_alpha_0.1111111111111111,0.614603
ridge_alpha_0.2222222222222222,0.614004
ridge_alpha_0.3333333333333333,0.613386
ridge_alpha_0.4444444444444444,0.612772
ridge_alpha_0.5555555555555556,0.612176
ridge_alpha_0.6666666666666666,0.611605


In [134]:
algorithme        = algos[nom_algo]